# **Sequential Chain**

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain, SequentialChain

In [ ]:
email = """
Betreff: Unerträgliche Verzögerung bei der Zustellung meiner Kreditkarte – Drittes Monat ohne Lösung!

Sehr geehrte Damen und Herren,

ich bin fassungslos und zutiefst enttäuscht über die absolute Inkompetenz, die Ihre Bank in Bezug auf die Zustellung meiner Kreditkarte an den Tag legt. Es sind nun fast drei Monate vergangen, seitdem ich die Karte beantragt habe, und ich habe sie immer noch nicht erhalten – und das trotz mehrfacher Zusicherungen Ihrerseits.

Was zur Hölle geht bei Ihnen vor? Drei Monate Wartezeit für eine simple Kreditkarte sind ein absolutes Unding und völlig unakzeptabel! Ich erwarte nicht nur eine sofortige Erklärung für diese erschreckende Verzögerung, sondern auch eine umgehende Lösung. Ihre Bank hat es in keiner Weise geschafft, meine Erwartung an einen einfachen Service zu erfüllen. So etwas habe ich noch nie erlebt und werde es auch nicht tolerieren!

Ich fordere Sie hiermit auf, mir innerhalb der nächsten 24 Stunden eine verbindliche Antwort zu geben, in der Sie mir den genauen Lieferstatus mitteilen und die schnellstmögliche Zustellung meiner Karte garantieren. Sollte dies nicht passieren, sehe ich mich gezwungen, sämtliche Konsequenzen zu ziehen und nicht nur meine Geschäftsbeziehung zu Ihrer Bank zu beenden, sondern auch rechtliche Schritte einzuleiten.

Ich erwarte eine umgehende Rückmeldung und erwarte, dass Sie Ihre Arbeit endlich auf die Reihe bekommen.

Mit wütenden Grüßen,
"""

In [ ]:
llm = ChatOpenAI(model='gpt-3.5-turbo', temperature=0)

In [ ]:
# First we will detect the language of the email
template_lang = "Return the language of the email. Only written the language the email was written in:\n {email}"
prompt_lang = ChatPromptTemplate.from_template(template_lang)
chain_lang = LLMChain(llm = llm, prompt=prompt_lang, output_key='language')

In [ ]:
# Translate the email into English language
template_translate = "Translate this {email} from {language} to English."
prompt_translate = ChatPromptTemplate.from_template(template_translate)
chain_translate = LLMChain(llm = llm, prompt=prompt_translate, output_key = 'translated_email')

In [ ]:
# Translate the email into English language
template_summary = """Create a short summary of the email: {translated_email}. 
The translation must highlight the key issues faced by the customer"""
prompt_summary = ChatPromptTemplate.from_template(template_summary)
chain_summary = LLMChain(llm = llm, prompt=prompt_summary, output_key = 'summary')

In [ ]:
seq_chain = SequentialChain(chains = [chain_lang, chain_translate, chain_summary],
                            input_variables=['email'],
                            output_variables=['language', 'translated_email', 'summary'],
                            verbose=False
                           )

In [ ]:
response = seq_chain(email)

In [ ]:
response.keys()

In [ ]:
response.get('language')

In [ ]:
print(response.get('translated_email'))

In [ ]:
print(response.get('summary'))

# **Output Parser**

In [ ]:
template_string = """You are a master branding consulatant who specializes in naming brands. \
You come up with catchy and memorable brand names.

Take the brand description below delimited by triple backticks and use it to create the name for a brand.

brand description: ```{brand_description}```

then based on the description and you hot new brand name give the brand a score 1-10 for how likely it is to succeed.

{format_instructions}
"""

In [ ]:
from langchain.prompts import ChatPromptTemplate
from langchain.output_parsers import ResponseSchema, StructuredOutputParser

In [ ]:
prompt = ChatPromptTemplate.from_template(template_string)

In [ ]:
brand_name_schema = ResponseSchema(name="brand_name",
                             description="This is the name of the brand")

likelihood_of_success_schema = ResponseSchema(name="likelihood_of_success",
                                      description="This is an integer score between 1-10")

reasoning_schema = ResponseSchema(name="reasoning",
                                    description="This is the reasons for the score")

In [ ]:
response_schemas = [brand_name_schema,
                    likelihood_of_success_schema,
                    reasoning_schema]

In [ ]:
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)
format_instructions = output_parser.get_format_instructions()
print(format_instructions)

In [ ]:
messages = prompt.format_messages(brand_description="a cool new sunglasses brand aimed at women",
                                format_instructions=format_instructions)

In [ ]:
print(messages[0].content)

In [ ]:
llm = ChatOpenAI(model='gpt-3.5-turbo', temperature=1.0)
response = llm.invoke(messages)

In [ ]:
response = output_parser.parse(response.content)

In [ ]:
response.keys()

In [ ]:
for key in response.keys():
    print(f"{key} : {response[key]}")

# **Conversation Summary Buffer Memory**

In [ ]:
from langchain.chains import ConversationChain
from langchain.memory import ConversationSummaryBufferMemory

In [ ]:
memory = ConversationSummaryBufferMemory(llm=llm, max_token_limit=100)

In [ ]:
conversation = ConversationChain(llm = llm, memory=memory)

In [ ]:
conversation.predict(input = 'Give me travel plans to San Francisco')

In [ ]:
conversation.predict(input = 'Give me some good places to visit while I am there')

In [ ]:
conversation.predict(input = 'How can I visit Hollywood while I am there')

In [ ]:
print(memory.load_memory_variables([])['history'])

# **Agent**

In [ ]:
!pip install -q wikipedia

In [ ]:
from langchain.agents import load_tools, initialize_agent, AgentType
from langchain.memory import ConversationBufferMemory

In [ ]:
agentic_memory = ConversationBufferMemory(memory_key = 'sl_ags_memory')
llm = ChatOpenAI(model = 'gpt-3.5-turbo',temperature=0)

In [ ]:
tools = load_tools(['llm-math','wikipedia'], llm=llm)

In [ ]:
agent = initialize_agent(tools, 
                         llm=llm, 
                         agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,
                         memory=agentic_memory,
                         handle_parsing_errors=True,
                         verbose=False
                        )

In [ ]:
agent.run(input = 'What is 2+2?')

In [ ]:
agent.run(input = 'What is 5 to the power of 3')

In [ ]:
agent.run(input = 'Who is Richard Feynman')